In [2]:
import sys
sys.path.append("../")

import backtrader as bt
from download import get_data
import datetime as dt
import talib as ta


In [3]:
INTERVAl = '4h'
SYMBOL = 'adausdt'
STARTTIME = dt.datetime(2017, 8, 31)
ENDTIME = dt.datetime.now()
IN_POSITION = False

data  = get_data(SYMBOL.upper(), INTERVAl, STARTTIME , ENDTIME)


In [4]:
data

,open,high,low,close,volume,close_time,quote_asset_volume,num_of_trades,taker_buy_base,taker_buy_quote
open_time,,,,,,,,,,
2018-04-17 04:00:00,0.25551,0.2880,0.24800,0.25698,29567756.16,2018-04-17 07:59:59.999,7.770296e+06,17419,11978707.89,3.146120e+06
2018-04-17 08:00:00,0.25636,0.2620,0.25229,0.26129,10304292.91,2018-04-17 11:59:59.999,2.638549e+06,7218,4720436.54,1.210593e+06
2018-04-17 12:00:00,0.26097,0.2666,0.25400,0.25570,10148285.89,2018-04-17 15:59:59.999,2.625652e+06,6895,3671178.28,9.524770e+05
2018-04-17 16:00:00,0.25570,0.2580,0.24800,0.25067,10342675.00,2018-04-17 19:59:59.999,2.615560e+06,6097,4624169.86,1.169097e+06
2018-04-17 20:00:00,0.25067,0.2550,0.23983,0.24260,7099283.28,2018-04-17 23:59:59.999,1.761766e+06,4939,3234172.82,8.032269e+05
...,...,...,...,...,...,...,...,...,...,...
2021-08-03 08:00:00,1.30110,1.3081,1.28550,1.29420,19805470.20,2021-08-03 11:59:59.999,2.568977e+07,89079,9868944.28,1.280292e+07
2021-08-03 12:00:00,1.29420,1.3675,1.28000,1.33850,79335492.86,2021-08-03 15:59:59.999,1.060034e+08,175836,39577819.08,5.288702e+07
2021-08-03 16:00:00,1.33850,1.3790,1.33490,1.36640,56210089.39,2021-08-03 19:59:59.999,7.645875e+07,272476,28548910.84,3.883878e+07


In [6]:
class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=50,  # period for the fast moving average
        pslow=200   # period for the slow moving average
    )

    def __init__(self):
        self.order = None
        sma1 = bt.ind.EMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.EMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                limit_price = self.datas[0].close
                size = float()
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position


            
    
# if __name__ == '__main__':
#     cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

#     # Create a data feed
#     data_feed = bt.feeds.PandasData(dataname=data)

#     cerebro.adddata(data_feed)  # Add the data feed
#     cerebro.broker.setcash(10)
#     cerebro.broker.setcommission(commission=0.001)
#     cerebro.addstrategy(SmaCross)  # Add the trading strategy
#     cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
#     result = cerebro.run()  # run it all


In [10]:
SmaCross.params.pfast

50

In [32]:
SmaCro

738005.3333333334

In [33]:
from datetime import datetime
datetime.fromtimestamp(a*1000)

datetime.datetime(1993, 5, 22, 0, 28, 53, 333333)

In [7]:
result

In [8]:
a

1.3385

In [9]:
A

NameError: name 'A' is not defined

In [ ]:
A

In [ ]:
A

In [ ]:
A

In [ ]:
A